# KEYWORDS
conjugate verbs
pluralize singular nouns

In [51]:
from word_forms.word_forms import get_word_forms

import json  

from nltk.stem import WordNetLemmatizer 
import nltk 
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer() 


In [30]:
get_word_forms("good")

{'n': {'good', 'goodness', 'goodnesses', 'goods'},
 'a': {'good'},
 'v': set(),
 'r': {'good', 'well'}}

In [50]:
help(get_word_forms)

Help on function get_word_forms in module word_forms.word_forms:

get_word_forms(word)
    args
        word : a word e.g "love"
    
    returns the related word forms corresponding to the input word. the output
    is a dictionary with four keys "n" (noun), "a" (adjective), "v" (verb)
    and "r" (adverb). The value for each key is a python Set containing
    related word forms with that part of speech.
    
    e.g. {'a': {'lovable', 'loveable'},
          'n': {'love', 'lover', 'lovers', 'loves'},
          'r': set(),
          'v': {'love', 'loved', 'loves', 'loving'}}



In [52]:
with open("unigramms_unique.json", "r") as f:
    data = json.load(f)

In [55]:
dct_mixed_words = {}
for el in data:
    dct_mixed_words[el['word']] = el['simlar_words']
dct_mixed_words['specifications']

['amplification',
 'recriminations',
 'intensification',
 'indemnification',
 'classification',
 'mortification',
 'quantification',
 'simplification',
 'qualifications',
 'magnification',
 'gratification',
 'reunification',
 'reciprocation',
 'specialization',
 'clarifications',
 'stratification',
 'qualification',
 'justifications',
 'objectification',
 'sophistication',
 'personification',
 'rectification',
 'fortifications',
 'certification',
 'clarification',
 'justification',
 'recrimination',
 'diversification',
 'notifications',
 'desertification',
 'simplifications',
 'fortification',
 'ramifications',
 'identification',
 'precipitation']

In [32]:
import re

regex = re.compile('[^a-zA-Z]')
regex.sub('', 'ab3d*E')

'abdE'

In [35]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [95]:
def extract_variants_from_dict(dct):
    wordforms_list = []
    for wordform_type in dct.keys():
        for wordform in dct[wordform_type]:
            print("wordform", wordform)
            if len(wordform.split()) == 1:
                wordforms_list.append(wordform)
    return set(wordforms_list)
def get_wordforms(word):
    MINIMAL_WORDFORMS_REQ = 6
    regex = re.compile('[^a-zA-Z]')
    clean_word = regex.sub('', word)
    print(clean_word)
    pos_tagged_ngramm = nltk.pos_tag([clean_word])
    for word_el in pos_tagged_ngramm:
        print(word_el)
        pos = get_wordnet_pos(word_el[1])
        if pos:
            lemma = lemmatizer.lemmatize(word_el[0], pos =pos)
        else:
            lemma = word_el[0]
        break
    if clean_word ==  lemma:
        wordforms_dict = get_word_forms(clean_word)
        print("wordforms_dict", wordforms_dict)
        wordform_set = extract_variants_from_dict(wordforms_dict)
        print("wordform_set", wordform_set)
    else:
        wordforms_dict = get_word_forms(clean_word)
        wordform_set = extract_variants_from_dict(wordforms_dict)
        
        wordforms_lemma_dict = get_word_forms(lemma)
        wordform_set_from_lemma = extract_variants_from_dict(wordforms_lemma_dict)
        wordform_set = wordform_set.union(wordform_set_from_lemma)

        
    if len(wordform_set) < MINIMAL_WORDFORMS_REQ:
        if lemma in dct_mixed_words:
            more_words_number = MINIMAL_WORDFORMS_REQ - len(wordform_set)
            add_words_list = dct_mixed_words[lemma][:more_words_number + 1]
            print("additional words", add_words_list)
            add_words_list = set(add_words_list)
            wordform_set = wordform_set.union(add_words_list)
        else:
            print("\nNOT_ENOUGH_WORDS_AND_NOT_ENOUGH_MIXED_WORDS",word)
    return wordform_set
get_wordforms("cossack")

cossack
('cossack', 'NN')
wordforms_dict {'n': set(), 'a': set(), 'v': set(), 'r': set()}
wordform_set set()

NOT_ENOUGH_WORDS_AND_NOT_ENOUGH_MIXED_WORDS cossack


set()

In [86]:
def extract_variants_from_dict(dct, original_word):
    wordforms_list = []
    for wordform_type in dct.keys():
        for wordform in dct[wordform_type]:
            #print(wordform.split()[0] == original_word)
            if len(wordform.split()) == 1 and wordform.split()[0] != original_word:
                #print(wordform)
                wordforms_list.append(wordform)
    return set(wordforms_list)

def get_wordforms(word):
    MINIMAL_WORDFORMS_REQ = 6
    regex = re.compile('[^a-zA-Z]')
    clean_word = regex.sub('', word)
    #print(clean_word)
    pos_tagged_ngramm = nltk.pos_tag([clean_word])
    for word_el in pos_tagged_ngramm:
        #print(word_el)
        pos = get_wordnet_pos(word_el[1])
        if pos:
            lemma = lemmatizer.lemmatize(word_el[0], pos =pos)
        else:
            lemma = word_el[0]
        break
    if clean_word ==  lemma:
        wordforms_dict = get_word_forms(clean_word)
        wordform_set = extract_variants_from_dict(wordforms_dict, clean_word)
    else:
        wordforms_dict = get_word_forms(clean_word)
        wordform_set = extract_variants_from_dict(wordforms_dict, clean_word)
        
        wordforms_lemma_dict = get_word_forms(lemma)
        wordform_set_from_lemma = extract_variants_from_dict(wordforms_lemma_dict, clean_word)
        wordform_set = wordform_set.union(wordform_set_from_lemma)

    if len(wordform_set) < MINIMAL_WORDFORMS_REQ:
        if lemma in dct_mixed_words:
            more_words_number = MINIMAL_WORDFORMS_REQ - len(wordform_set)
            add_words_list = dct_mixed_words[lemma][:more_words_number + 1]
            print("additional words", add_words_list)
            add_words_list = set(add_words_list)
            wordform_set = wordform_set.union(add_words_list)
        else:
            print("NOT_ENOUGH_WORDS_AND_NOT_ENOUGH_MIXED_WORDS",word)
    return wordform_set
get_wordforms("machine")

{'machined',
 'machineries',
 'machinery',
 'machines',
 'machining',
 'machinist',
 'machinists'}